# Exporting Earth Engine Data

```{contents}
:local:
:depth: 2
```

## Introduction

## Technical requirements

```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
```

```bash
jupyter lab
```

In [ ]:
# %pip install pygis

In [1]:
import ee
import geemap

In [2]:
geemap.ee_initialize()

## Exporting images

In [3]:
Map = geemap.Map()

image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318').select(
    ['B5', 'B4', 'B3']
)

vis_params = {'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}

Map.centerObject(image, 8)
Map.addLayer(image, vis_params, 'Landsat')
Map

Map(center=[37.47164678275328, -122.14450014746849], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
region = ee.Geometry.BBox(-122.5955, 37.5339, -122.0982, 37.8252)
fc = ee.FeatureCollection(region)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fc.style(**style), {}, 'ROI')
Map

Map(bottom=25701.0, center=[37.470498470798724, -122.14324951171876], controls=(WidgetControl(options=['positi…

### To local drive

Algunas claves de esta funcionalidad: 

* Limitación importante para exportarlo directamente a local -> solo 32mb...
* La función download_ee_image() permite descargar cada porción de 32mb y fusionarla de nuevo para poder obetenr la imagen completa
* fishnet es util sobre todo cuanto trabajas con imagenes muy grandes. Por ejemplo, mapa global. Divide la imagen en tiles de una fishnet y los va descargando uno a uno --> Luego lo puedes unir con GDAL o con un GIS
* **¿Y por qué no directamente bajarlo de GEE?** porque por el método oficial toca enviar la petición y esperar al resultado -> puede tardar minutos o horas, depende el tamaño del file

In [8]:
geemap.ee_export_image(image, filename="landsat.tif", scale=30, region=region) # reduce a la región

Generating URL ...
Please wait ...
Data downloaded to c:\Users\skype\VS-python-codes\practicas-GEE\gee\landsat.tif


In [9]:
projection = image.select(0).projection().getInfo()
projection

{'type': 'Projection',
 'crs': 'EPSG:32610',
 'transform': [30, 0, 460785, 0, -30, 4264215]}

In [11]:
crs = projection['crs']
crs_transform = projection['transform']

In [12]:
# Especificar proyección
geemap.ee_export_image(
    image,
    filename="landsat_crs.tif",
    crs=crs,
    crs_transform=crs_transform,
    region=region,
)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\skype\VS-python-codes\practicas-GEE\gee\landsat_crs.tif


In [13]:
# Descaga la imagen completa
geemap.download_ee_image(image, filename='landsat_full.tif', scale=60)

landsat_full.tif: |          | 0.00/179M (raw) [  0.0%] in 00:00 (eta:     ?)

In [14]:
# Divide la imagen en una fishnet
fishnet = geemap.fishnet(image.geometry(), rows=4, cols=4, delta=0.5)
style = {'color': 'ffff00ff', 'fillColor': '00000000'}
Map.addLayer(fishnet.style(**style), {}, 'Fishnet')
Map

Map(bottom=51154.0, center=[37.35705927979369, -122.10754394531251], controls=(WidgetControl(options=['positio…

In [ ]:
out_dir = 'Downloads'
geemap.download_ee_image_tiles(
    image, fishnet, out_dir, prefix="landsat_", crs="EPSG:3857", scale=30
)

Para una descarga más rápida también se puede usar **download_ee_image_tiles_parallel()**, que paraleliza la descarga en vez de ejecutarlo secuencia

In [ ]:
out_dir = 'Downloads'
geemap.download_ee_image_tiles_parallel(
    image, fishnet, out_dir, prefix="landsat_", crs="EPSG:3857", scale=30
)

### To Google Drive

In [15]:
geemap.ee_export_image_to_drive(
    image, description='landsat', folder='export', region=region, scale=30
)

### To Asset

In [ ]:
assetId = 'landsat_sfo'
geemap.ee_export_image_to_asset(
    image, description='landsat', assetId=assetId, region=region, scale=30
)

### To Cloud Storage

In [ ]:
bucket = 'your-bucket'
geemap.ee_export_image_to_cloud_storage(
    image, description='landsat', bucket=None, region=region, scale=30
)

### To NumPy array

In [ ]:
region = ee.Geometry.BBox(-122.5003, 37.7233, -122.3410, 37.8026)
rgb_img = geemap.ee_to_numpy(image, region=region)

In [ ]:
print(rgb_img.shape)

In [ ]:
import matplotlib.pyplot as plt

rgb_img_test = (255 * ((rgb_img[:, :, 0:3]) + 0.2)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()

## Exporting image collections

Anteriormente vimos como exportar una sola, pero **es posible importar una colección de imágenes** --> util por ejemplo si estamos trabajando con **series temporales** y no tenemos especial interés en una imagen

In [5]:
point = ee.Geometry.Point(-99.2222, 46.7816)
collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(point)
    .filterDate('2008-01-01', '2018-01-01')
    .filter(ee.Filter.listContains("system:band_names", "N"))
)

In [6]:
# ID específico de cada img
collection.aggregate_array('system:index').getInfo()

['m_4609915_sw_14_1_20090818',
 'm_4609915_sw_14_1_20100629',
 'm_4609915_sw_14_1_20120714',
 'm_4609915_sw_14_1_20140901',
 'm_4609915_sw_14_1_20150926',
 'm_4609915_sw_14_h_20160704',
 'm_4609915_sw_14_h_20170703']

### To local drive

In [ ]:
out_dir = 'Downloads'
geemap.ee_export_image_collection(collection, out_dir=out_dir, scale=10)

### To Google Drive

In [ ]:
geemap.ee_export_image_collection_to_drive(collection, folder='export', scale=10)

### To Assets

In [ ]:
# Para guardarlo como asset y poder seguir utilizandolo en la cuenta de GEE
geemap.ee_export_image_collection_to_asset(collection, scale=10)

### Ejemplo 1: series temporales con imágenes del NAIP

Ejemplo de un workflow que propone en el video para **descargar la serie temporal del National Agriculture Imagery Program (NAIP) de un roi** que definimos de forma interactiva:

In [4]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# función para convertir a ts las imagenes del NAIP
ts = geemap.naip_timeseries(m.user_roi)


m = geemap.Map()
image = ts.first()
m.add_layer(image)
m.center_object(image)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Ejemplo 2: series temporales de imágenes Landsat

Es posible aplicar el mismo flujo para **cualquier colección de imágenes** --> aplicando la función **create_timeseries()**
* Revisar la doc oficial de la función para ver todos los parámetros
* En el parámetro de Collection se le puede pasar cualquiera que hubiese filtrado previamente
* La frecuencia permite especificar el número de imágenes totales (por ejemplo: 1 al año o 1 al mes) y el parámetro reducer especifica como se pasará de varias imágenes a una única por cada periodo temporal
* El parámetro drop enty establece como actuar en caso de que no haya imágenes en un año (aplicar la media, por ejemplo)

In [39]:
# Load borders
borders = ee.FeatureCollection("FAO/GAUL/2015/level1")
region = borders.filter(ee.Filter.eq('ADM1_NAME', 'Castilla y León'))

# Load Sentinel 2 data
collection = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate('2021-01-01', '2022-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
)

start_date = '2021-01-01'
end_date = '2022-01-01'

In [40]:
ts2 = geemap.create_timeseries(collection, start_date, end_date, region, frequency = "month",
                               reducer = 'mean', bands = ['B4', 'B3', 'B2'])

In [41]:
ts2

In [44]:
m = geemap.Map()
image = ts2.first()
vis = {
    'min': 0.0,
    'max': 10000,
    'bands': ['B4', 'B3', 'B2'],
}

m.add_layer(image)
# m.center_object(image)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Exporting videos

Sirve para exportar gif animados. Por el momento no está disponible para descargarlo en local. Toca exportarlo a Drive. 

La función **create_timelapse()** también puede utilizarse en estos casos

In [45]:
collection = (
    ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')
    .filter(ee.Filter.eq('WRS_PATH', 44))
    .filter(ee.Filter.eq('WRS_ROW', 34))
    .filter(ee.Filter.lt('CLOUD_COVER', 30))
    .filterDate('1991-01-01', '2011-12-30')
    .select(['B4', 'B3', 'B2'])
    .map(lambda img: img.multiply(512).uint8())
)
region = ee.Geometry.Rectangle([-122.7286, 37.6325, -122.0241, 37.9592])

In [46]:
geemap.ee_export_video_to_drive(
    collection, folder='export', framesPerSecond=12, dimensions=720, region=region
)

Exporting myExportVideoTask... Please check the Task Manager from the JavaScript Code Editor.


## Exporting image thumbnails

Permite descargar una miniatura o jpg a alta resolución. Es poco pesada y puede descargarse rapidamente. Es una solución intermedia entre bajarse el TIF y hacer una captura. 

A partir de estas miniatura a alta resolución es posible **crer un gif animado** como haciamos antes pero con MUCHA mayor calidad -> opción interesante para publicaciones.

In [47]:
roi = ee.Geometry.Point([-122.44, 37.75])
collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
    .filterBounds(roi)
    .sort("CLOUD_COVER")
    .limit(10) # Selecciona el top 10 con menos nubes
)

image = collection.first()

In [48]:
Map = geemap.Map()

vis_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.3,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(image, vis_params, "LANDSAT 8")
Map.setCenter(-122.44, 37.75, 8)
Map

Map(center=[37.75, -122.44], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

In [49]:
out_img = 'landsat.jpg'
region = ee.Geometry.BBox(-122.5955, 37.5339, -122.0982, 37.8252)
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=1000, region=region)

In [50]:
geemap.show_image(out_img)

Output()

In [51]:
# En este paso descarga las 10 imágenes de la colección que filtramos anteriormente-> aquellas 10 con menos nubes
out_dir = 'Downloads'
geemap.get_image_collection_thumbnails(
    collection,
    out_dir,
    vis_params,
    dimensions=1000,
    region=region,
)

Total number of images: 10



Si quieremos crear un gif a partir de esas 10 imágenes de thumbnail utilizamos geemap.make_gif() -> revisar opciones de configuración. 

In [53]:
geemap.make_gif("Downloads", 'landsat.gif', fps=3)

### Ejemplo creando un gif del Panta de Sau (Cataluña)

In [31]:
# Filtramos la Colección con los últimos dos años donde se ha producido sequía
roi = ee.Geometry.Point([2.35648292785471, 41.98895765]) # buscado en Google
collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
    .filterBounds(roi)
    .filterDate('2023-01-01', '2024-04-16')
    .sort("CLOUD_COVER")
    .limit(10) # Selecciona el top 10 con menos nubes
)

In [32]:
# Ordenamos por fecha
collection = collection.sort('DATE_ACQUIRED')

image = collection.first()

In [10]:
Map = geemap.Map()

vis_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.3,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(image, vis_params, "LANDSAT 8")
Map.setCenter(2.35648292785471, 41.98895765, 13)
Map

Map(center=[41.98895765, 2.35648292785471], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…

In [11]:
if Map.user_roi is not None:
    print(Map.user_roi.getInfo())

{'geodesic': False, 'type': 'Polygon', 'coordinates': [[[2.331047, 41.948086], [2.331047, 42.007171], [2.436104, 42.007171], [2.436104, 41.948086], [2.331047, 41.948086]]]}


In [12]:
# Obtenemos las coordenadas del rectangulo dibujado en el pantano
Map.user_roi_coords()

[2.331, 41.9481, 2.4361, 42.0072]

In [35]:
# Descargamos la primera miniatura de prueba
out_img = 'landsat_pantano.jpg'
region = ee.Geometry.BBox(2.331, 41.9481, 2.4361, 42.0072)
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=1000, region=region)

In [37]:
# En este paso descarga las 10 imágenes de la colección que filtramos anteriormente-> aquellas 10 con menos nubes
out_dir = 'Downloads_pantano'
geemap.get_image_collection_thumbnails(
    collection,
    out_dir,
    vis_params,
    dimensions=1000,
    region=region,
)

Total number of images: 10



In [38]:
# Cramos el gif del pantano pasandole el directorio
geemap.make_gif("Downloads_pantano", 'landsat_pantano.gif', fps=3)

In [39]:
geemap.show_image('landsat_pantano.gif')

Output()

## Exporting feature collections

Para exportar datos vectoriales en varios formatos

In [ ]:
Map = geemap.Map()
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('wld_rgn', 'Europe')
)

Map.addLayer(fc, {}, "Europe")
Map.centerObject(fc, 3)
Map

### To local drive

In [ ]:
geemap.ee_to_shp(fc, filename='europe.shp', selectors=None)

In [ ]:
geemap.ee_export_vector(fc, filename='europe2.shp')

In [ ]:
geemap.ee_to_geojson(fc, filename='europe.geojson')

In [ ]:
geemap.ee_to_csv(fc, filename='europe.csv')

In [ ]:
gdf = geemap.ee_to_gdf(fc)
gdf

In [ ]:
df = geemap.ee_to_df(fc)
df

### To Google Drive

In [ ]:
geemap.ee_export_vector_to_drive(
    fc, description="europe", fileFormat='SHP', folder="export"
)

### To Asset

In [ ]:
geemap.ee_export_vector_to_asset(fc, description='Exporting Europe', assetId='europe')

## Exporting maps

GEE no es recomendable para la producción cartográfica. No obstante, existe la opción de exportarlo como HTML. 

IMPORTANTE: el link del HTML dura solo 24 h -> si quieres que sea permanente tienes que crear la web app y que automaticamente cada 24 h regenere el layer

In [ ]:
Map = geemap.Map()
image = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(image, vis_params, 'SRTM DEM', True)
Map

In [ ]:
Map.to_html(
    filename="mymap.html", title="Earth Engine Map", width='100%', height='800px'
)

## Using the high-volume endpoint

Permite descargar cientos de imágenes en minutos. Especialmente en útil para Deep Learning (DL). Utiliza batch processing. 

In [ ]:
import ee
import geemap
import logging
import multiprocessing
import os
import requests
import shutil
from retry import retry

In [ ]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
region = Map.user_roi

if region is None:
    region = ee.Geometry.Polygon(
        [
            [
                [-122.513695, 37.707998],
                [-122.513695, 37.804359],
                [-122.371902, 37.804359],
                [-122.371902, 37.707998],
                [-122.513695, 37.707998],
            ]
        ],
        None,
        False,
    )

In [ ]:
image = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(region)
    .filterDate('2020', '2021')
    .mosaic()
    .clip(region)
    .select('N', 'R', 'G')
)

In [ ]:
Map = geemap.Map()
Map.addLayer(image, {}, "Image")
Map.addLayer(region, {}, "ROI", False)
Map.centerObject(region, 12)
Map

In [ ]:
out_dir = 'Downloads'
params = {
    'count': 1000,  # How many image chips to export
    'buffer': 127,  # The buffer distance (m) around each point
    'scale': 100,  # The scale to do stratified sampling
    'seed': 1,  # A randomization seed to use for subsampling.
    'dimensions': '256x256',  # The dimension of each image chip
    'format': "png",  # The output image format, can be png, jpg, ZIPPED_GEO_TIFF, GEO_TIFF, NPY
    'prefix': 'tile_',  # The filename prefix
    'processes': 25,  # How many processes to used for parallel processing
    'out_dir': out_dir,  # The output directory. Default to the current working directly
}

In [ ]:
def getRequests():
    img = ee.Image(1).rename("Class").addBands(image)
    points = img.stratifiedSample(
        numPoints=params['count'],
        region=region,
        scale=params['scale'],
        seed=params['seed'],
        geometries=True,
    )
    Map.data = points
    return points.aggregate_array('.geo').getInfo()

In [ ]:
@retry(tries=10, delay=1, backoff=2)
def getResult(index, point):
    point = ee.Geometry.Point(point['coordinates'])
    region = point.buffer(params['buffer']).bounds()

    if params['format'] in ['png', 'jpg']:
        url = image.getThumbURL(
            {
                'region': region,
                'dimensions': params['dimensions'],
                'format': params['format'],
            }
        )
    else:
        url = image.getDownloadURL(
            {
                'region': region,
                'dimensions': params['dimensions'],
                'format': params['format'],
            }
        )

    if params['format'] == "GEO_TIFF":
        ext = 'tif'
    else:
        ext = params['format']

    r = requests.get(url, stream=True)
    if r.status_code != 200:
        r.raise_for_status()

    out_dir = os.path.abspath(params['out_dir'])
    basename = str(index).zfill(len(str(params['count'])))
    filename = f"{out_dir}/{params['prefix']}{basename}.{ext}"
    with open(filename, 'wb') as out_file:
        shutil.copyfileobj(r.raw, out_file)
    print("Done: ", basename)

In [ ]:
%%time
logging.basicConfig()
items = getRequests()

pool = multiprocessing.Pool(params['processes'])
pool.starmap(getResult, enumerate(items))

pool.close()

In [ ]:
Map.addLayer(Map.data, {}, "Sample points")
Map

In [ ]:
geemap.ee_to_shp(Map.data, filename='points.shp')

## Summary